In [1]:
import sys
import json
import gzip
from configparser import ConfigParser
import psycopg2

VERBOSE            = True
CLUSTERS           = []

def config(configname='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(configname)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, configname))

    return db

conn_pg = None

def connect_pg(configname='database.ini'):
    """ Connect to the PostgreSQL database server """
    try:
        # read connection parameters
        params = config(configname=configname)

        # connect to the PostgreSQL server
        if VERBOSE:
            print('Connecting to the PostgreSQL database...')
        global conn_pg
        conn_pg = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn_pg.cursor()
        
	# execute a statement
        if VERBOSE:
            print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        if VERBOSE:
            print(db_version)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        exit() # pretty fatal


def close_pg():
    """ Close connection to the PostgreSQL database server """
    try:
        if conn_pg is not None:
            conn_pg.close()
            if VERBOSE:
                print('Database connection closed.')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def execute_pg(sql_stat=None):
    """ Execute single command on the PostgreSQL database server """
    try:
        cur = conn_pg.cursor()
        cur.execute(sql_stat)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [2]:
INDEX = 0
COUNTER = 1
TRACKER = 0

sqlbase = 'SELECT DISTINCT cluster_id FROM TABLEBASE;'

sqlsentence = "UPDATE TABLEPROB SET _sentence=INSERTHERE WHERE index='INDEX';"

sqldict = "UPDATE _dict SET dict=add(dict, "

sqldictadd1 = "xREPLACE=1:0.5;"

sqldictadd2 = "xREPLACE=2:0.5;"

sqldictadd3 = "yREPLACE=1:0.5;"

sqldictadd4 = "yREPLACE=2:0.5;"

sqldictadd5 = "xREPLACE=NUMB:PROB;"

sqldictadd6 = "yREPLACE=NUMB:PROB;"

sqldictadd7 = "zREPLACE=NUMB:PROB;"

sqldictadd8 = "wREPLACE=NUMB:PROB;"

sqldictadd9 = "vREPLACE=NUMB:PROB;"

sentencedefault = "Bdd('xREPLACE=1')"

sentencediff1 = "Bdd('xREPLACE=2')"

sentencediff2= "Bdd('xREPLACE=2 & yREPLACE=1')"

sentencediff3= "Bdd('xREPLACE=2 & yREPLACE=2')"

sentencediff4 = "Bdd('xREPLACE=1 | xREPLACE=2')"

sentencediff5 = "Bdd('xREPLACE=NUMUNO | xREPLACE=NUMDOS')"

sentencediff6 = "Bdd('xREPLACE=NUMUNO | xREPLACE=NUMDOS | xREPLACE=NUMTRES')"

sentencediff7 = "Bdd('xREPLACE=NUMUNO | xREPLACE=NUMDOS | xREPLACE=NUMTRES | xREPLACE=NUMQUAT')"

sentencediff8 = "Bdd('xREPLACE=NUMUNO | xREPLACE=NUMDOS | xREPLACE=NUMTRES | xREPLACE=NUMQUAT | xREPLACE=NUMQUIN')"

sentencediff9 = "Bdd('xREPLACE=NUMUNO & yREPLACE=NUMDOS')"

sentencediff10 = "Bdd('xREPLACE=NUMUNO & zREPLACE=NUMDOS')"

sentencediff11 = "Bdd('xREPLACE=NUMUNO & wREPLACE=NUMDOS')"

sentencediff12 = "Bdd('xREPLACE=NUMUNO & vREPLACE=NUMDOS')"

sqlgetcluster= "SELECT * FROM TABLEBASE WHERE cluster_id=CLUSTNUM;"

insertdefault = "INSERT INTO TABLEPROB VALUES (INDEX, NEWID, CLUSTNUM, 'CATEGORY', PRICE, SENTENCE);"

createdict = "DELETE FROM _dict WHERE name='DICTNAME'; INSERT INTO _dict(name,dict) VALUES ('DICTNAME',dictionary(''));"

def getIDs(sql, base):
    global CLUSTERS
    CLUSTERS = []
    try:
        execute = sql.replace('TABLEBASE', base)
        cur = conn_pg.cursor()
        cur.execute(execute)
        result = cur.fetchall()
        for i in result:
            CLUSTERS.append(i[0])
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
            
def completeTable2(prob, base, dictname):
    global INDEX, COUNTER, TRACKER
    INDEX = 0
    COUNTER = 1
    TRACKER = 0
    try:
        startdict = createdict.replace('DICTNAME', dictname)
        doSQL(startdict)
        finaldict = sqldict + "'"
        for i in CLUSTERS:
            cluster = getCluster(i, base)
            sentencebasic = sentencedefault.replace('REPLACE', str(cluster[0][1]))
            sqlbasic = insertdefault.replace('TABLEPROB', prob)
            sqlbasic = sqlbasic.replace('CATEGORY', cluster[0][0])
            sqlbasic = sqlbasic.replace('CLUSTNUM', str(cluster[0][1]))
            sql1 = sqlbasic.replace('INDEX', str(INDEX))
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql1 = sql1.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[0][3] == None):
                sql1 = sql1.replace('PRICE', 'NULL')
            else:
                sql1 = sql1.replace('PRICE', "'"+str(cluster[0][3])+"'")
            sql1 = sql1.replace('SENTENCE', sentencebasic)
            #print(sql1)
            #print('\n')
            sql2 = sqlbasic.replace('INDEX', str(INDEX))
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql2 = sql2.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[1][3] == None):
                sql2 = sql2.replace('PRICE', 'NULL')
            else:
                sql2 = sql2.replace('PRICE', "'"+str(cluster[1][3])+"'")
            sql2 = sql2.replace('SENTENCE', sentencebasic)
            #print(sql2)
            #print('\n')
            doSQL(sql1)
            doSQL(sql2)
            temp = sqldictadd1.replace('REPLACE', str(cluster[0][1]))
            finaldict = finaldict + temp
            if (cluster[0][0] != cluster[1][0]):
                print("THE THING THAT WASN'T SUPPOSED TO HAPPEN HAPPENED AT:")
                print(cluster[0][1])
            if (cluster[0][3] == cluster[1][3]):
                sentence = sentencediff4.replace('REPLACE', str(cluster[0][1]))
                execute = sqlsentence.replace('TABLEPROB', prob)
                execute = execute.replace('INSERTHERE', sentence)
                execute = execute.replace('INDEX', str(INDEX-2))
                #print(execute)
                #print('\n')
                doSQL(execute)
                temp = sqldictadd2.replace('REPLACE', str(cluster[0][1]))
                finaldict = finaldict + temp
            else:
                if (cluster[0][3] == None):
                    sentence = sentencediff4.replace('REPLACE', str(cluster[0][1]))
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(INDEX-1))
                    #print(execute)
                    #print('\n')
                    doSQL(execute)
                    temp = sqldictadd2.replace('REPLACE', str(cluster[1][1]))
                    finaldict = finaldict + temp
                elif (cluster[1][3] == None):
                    sentence = sentencediff4.replace('REPLACE', str(cluster[0][1]))
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(INDEX-2))
                    #print(execute)
                    #print('\n')
                    doSQL(execute)
                    temp = sqldictadd2.replace('REPLACE', str(cluster[0][1]))
                    finaldict = finaldict + temp
                else:
                    sentence1 = sentencediff2.replace('REPLACE', str(cluster[0][1]))
                    sentence2 = sentencediff3.replace('REPLACE', str(cluster[0][1]))
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    #print(execute1)
                    #print('\n')
                    doSQL(execute1)
                    #print(execute2)
                    #print('\n')
                    doSQL(execute2)
                    temp1 = sqldictadd2.replace('REPLACE', str(cluster[0][1]))
                    temp2 = sqldictadd3.replace('REPLACE', str(cluster[0][1]))
                    temp3 = sqldictadd4.replace('REPLACE', str(cluster[0][1]))
                    finaldict = finaldict + temp1 + temp2 + temp3
            if (TRACKER > 100):
                TRACKER = 0
                sys.stdout.write('.')
                sys.stdout.flush()
        finaldict = finaldict + "') WHERE name='DICTNAME';"
        finaldict = finaldict.replace('DICTNAME', dictname)
        #print(finaldict)
        #print('\n')
        doSQL(finaldict)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
        
def getCluster(clusterID, table):
    try:
        execute = sqlgetcluster.replace('TABLEBASE', table)
        execute = execute.replace('CLUSTNUM', str(clusterID))
        cur = conn_pg.cursor()
        cur.execute(execute)
        result = cur.fetchall()
        cur.close()
        return result
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def doSQL(sql):
    try:
        cur = conn_pg.cursor()
        cur.execute(sql)
        conn_pg.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [3]:
sentencexs7 = "Bdd('xREPLACE=NUM1 | xREPLACE=NUM2 | xREPLACE=NUM3 | xREPLACE=NUM4 | xREPLACE=NUM5 | xREPLACE=NUM6 | xREPLACE=NUM7')"

sentencexs10 = "Bdd('xREPLACE=NUM1 | xREPLACE=NUM2 | xREPLACE=NUM3 | xREPLACE=NUM4 | xREPLACE=NUM5 | xREPLACE=NUM6 | xREPLACE=NUM7 | xREPLACE=NUM8 | xREPLACE=NUM9 | xREPLACE=NUM10')"

sentencexs15 = "Bdd('xREPLACE=NUM1 | xREPLACE=NUM2 | xREPLACE=NUM3 | xREPLACE=NUM4 | xREPLACE=NUM5 | xREPLACE=NUM6 | xREPLACE=NUM7 | xREPLACE=NUM8 | xREPLACE=NUM9 | xREPLACE=NUM10 | xREPLACE=NUM11 | xREPLACE=NUM12 | xREPLACE=NUM13 | xREPLACE=NUM14 | xREPLACE=NUM15')"

def sentenceAll(clusters, prob, index):
    try:
        sentence = sentencexs15.replace('REPLACE', str(clusters[0][1]))
        sentence = sentence.replace('NUM1', '1')
        sentence = sentence.replace('NUM2', '2')
        sentence = sentence.replace('NUM3', '3')
        sentence = sentence.replace('NUM4', '4')
        sentence = sentence.replace('NUM5', '5')
        sentence = sentence.replace('NUM6', '6')
        sentence = sentence.replace('NUM7', '7')
        sentence = sentence.replace('NUM8', '8')
        sentence = sentence.replace('NUM9', '9')
        sentence = sentence.replace('NUM10', '10')
        sentence = sentence.replace('NUM11', '11')
        sentence = sentence.replace('NUM12', '12')
        sentence = sentence.replace('NUM13', '13')
        sentence = sentence.replace('NUM14', '14')
        sentence = sentence.replace('NUM15', '15')
        execute = sqlsentence.replace('TABLEPROB', prob)
        execute = execute.replace('INSERTHERE', sentence)
        execute = execute.replace('INDEX', str(index))
        doSQL(execute)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
def sentenceSeven(clusters, prob, index, one, two, three, four, five, six, seven):
    try:
        #print(clusters[0][1])
        sentence = sentencexs7.replace('REPLACE', str(clusters[0][1]))
        sentence = sentence.replace('NUM1', str(one))
        sentence = sentence.replace('NUM2', str(two))
        sentence = sentence.replace('NUM3', str(three))
        sentence = sentence.replace('NUM4', str(four))
        sentence = sentence.replace('NUM5', str(five))
        sentence = sentence.replace('NUM6', str(six))
        sentence = sentence.replace('NUM7', str(seven))
        execute = sqlsentence.replace('TABLEPROB', prob)
        execute = execute.replace('INSERTHERE', sentence)
        execute = execute.replace('INDEX', str(index))
        doSQL(execute)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
def sentenceTen(clusters, prob, index, one, two, three, four, five, six, seven, eight, nine, ten):
    try:
        sentence = sentencexs10.replace('REPLACE', str(clusters[0][1]))
        sentence = sentence.replace('NUM1', str(one))
        sentence = sentence.replace('NUM2', str(two))
        sentence = sentence.replace('NUM3', str(three))
        sentence = sentence.replace('NUM4', str(four))
        sentence = sentence.replace('NUM5', str(five))
        sentence = sentence.replace('NUM6', str(six))
        sentence = sentence.replace('NUM7', str(seven))
        sentence = sentence.replace('NUM8', str(eight))
        sentence = sentence.replace('NUM9', str(nine))
        sentence = sentence.replace('NUM10', str(ten))
        execute = sqlsentence.replace('TABLEPROB', prob)
        execute = execute.replace('INSERTHERE', sentence)
        execute = execute.replace('INDEX', str(index))
        doSQL(execute)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def twoDiff(clusters, prob, clus1, clus2, xval):
    global INDEX, COUNTER, TRACKER
    try:
        executebase = insertdefault.replace('TABLEPROB', prob)
        executebase = executebase.replace('CATEGORY', clusters[0][0])
        executebase = executebase.replace('CLUSTNUM', str(clusters[0][1]))
        executebase = executebase.replace('NEWID', str(COUNTER))
        COUNTER = COUNTER + 1
        execute1 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute2 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute1 = execute1.replace('PRICE', "'"+str(clusters[clus1][3])+"'")
        execute2 = execute2.replace('PRICE', "'"+str(clusters[clus2][3])+"'")
        sentencebase = sentencediff9.replace('REPLACE', str(clusters[0][1]))
        sentencebase = sentencebase.replace('NUMUNO', str(xval))
        sentence1 = sentencebase.replace('NUMDOS', '1')
        sentence2 = sentencebase.replace('NUMDOS', '2')
        execute1 = execute1.replace('SENTENCE', sentence1)
        execute2 = execute2.replace('SENTENCE', sentence2)
        doSQL(execute1)
        doSQL(execute2)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def twoThirdDiff(clusters, prob, clus1, clus2, xval):
    global INDEX, COUNTER, TRACKER
    try:
        executebase = insertdefault.replace('TABLEPROB', prob)
        executebase = executebase.replace('CATEGORY', clusters[0][0])
        executebase = executebase.replace('CLUSTNUM', str(clusters[0][1]))
        executebase = executebase.replace('NEWID', str(COUNTER))
        COUNTER = COUNTER + 1
        execute1 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute2 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute1 = execute1.replace('PRICE', "'"+str(clusters[clus1][3])+"'")
        execute2 = execute2.replace('PRICE', "'"+str(clusters[clus2][3])+"'")
        sentencebase = sentencediff10.replace('REPLACE', str(clusters[0][1]))
        sentencebase = sentencebase.replace('NUMUNO', str(xval))
        sentence1 = sentencebase.replace('NUMDOS', '1')
        sentence2 = sentencebase.replace('NUMDOS', '2')
        execute1 = execute1.replace('SENTENCE', sentence1)
        execute2 = execute2.replace('SENTENCE', sentence2)
        doSQL(execute1)
        doSQL(execute2)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def twoQuartDiff(clusters, prob, clus1, clus2, xval):
    global INDEX, COUNTER, TRACKER
    try:
        executebase = insertdefault.replace('TABLEPROB', prob)
        executebase = executebase.replace('CATEGORY', clusters[0][0])
        executebase = executebase.replace('CLUSTNUM', str(clusters[0][1]))
        executebase = executebase.replace('NEWID', str(COUNTER))
        COUNTER = COUNTER + 1
        execute1 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute2 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute1 = execute1.replace('PRICE', "'"+str(clusters[clus1][3])+"'")
        execute2 = execute2.replace('PRICE', "'"+str(clusters[clus2][3])+"'")
        sentencebase = sentencediff11.replace('REPLACE', str(clusters[0][1]))
        sentencebase = sentencebase.replace('NUMUNO', str(xval))
        sentence1 = sentencebase.replace('NUMDOS', '1')
        sentence2 = sentencebase.replace('NUMDOS', '2')
        execute1 = execute1.replace('SENTENCE', sentence1)
        execute2 = execute2.replace('SENTENCE', sentence2)
        doSQL(execute1)
        doSQL(execute2)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def threeThirdDiff(clusters, prob, clus1, clus2, clus3, xval):
    global INDEX, COUNTER, TRACKER
    try:
        executebase = insertdefault.replace('TABLEPROB', prob)
        executebase = executebase.replace('CATEGORY', clusters[0][0])
        executebase = executebase.replace('CLUSTNUM', str(clusters[0][1]))
        executebase = executebase.replace('NEWID', str(COUNTER))
        COUNTER = COUNTER + 1
        execute1 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute2 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute3 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute1 = execute1.replace('PRICE', "'"+str(clusters[clus1][3])+"'")
        execute2 = execute2.replace('PRICE', "'"+str(clusters[clus2][3])+"'")
        execute3 = execute3.replace('PRICE', "'"+str(clusters[clus3][3])+"'")
        sentencebase = sentencediff11.replace('REPLACE', str(clusters[0][1]))
        sentencebase = sentencebase.replace('NUMUNO', str(xval))
        sentence1 = sentencebase.replace('NUMDOS', '1')
        sentence2 = sentencebase.replace('NUMDOS', '2')
        sentence3 = sentencebase.replace('NUMDOS', '3')
        execute1 = execute1.replace('SENTENCE', sentence1)
        execute2 = execute2.replace('SENTENCE', sentence2)
        execute3 = execute3.replace('SENTENCE', sentence3)
        doSQL(execute1)
        doSQL(execute2)
        doSQL(execute3)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def threeHalfDiff(clusters, prob, clus1, clus2, clus3, xval):
    global INDEX, COUNTER, TRACKER
    try:
        executebase = insertdefault.replace('TABLEPROB', prob)
        executebase = executebase.replace('CATEGORY', clusters[0][0])
        executebase = executebase.replace('CLUSTNUM', str(clusters[0][1]))
        executebase = executebase.replace('NEWID', str(COUNTER))
        COUNTER = COUNTER + 1
        execute1 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute2 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute3 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute1 = execute1.replace('PRICE', "'"+str(clusters[clus1][3])+"'")
        execute2 = execute2.replace('PRICE', "'"+str(clusters[clus2][3])+"'")
        execute3 = execute3.replace('PRICE', "'"+str(clusters[clus3][3])+"'")
        sentencebase = sentencediff12.replace('REPLACE', str(clusters[0][1]))
        sentencebase = sentencebase.replace('NUMUNO', str(xval))
        sentence1 = sentencebase.replace('NUMDOS', '1')
        sentence2 = sentencebase.replace('NUMDOS', '2')
        sentence3 = sentencebase.replace('NUMDOS', '3')
        execute1 = execute1.replace('SENTENCE', sentence1)
        execute2 = execute2.replace('SENTENCE', sentence2)
        execute3 = execute3.replace('SENTENCE', sentence3)
        doSQL(execute1)
        doSQL(execute2)
        doSQL(execute3)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def fourQaurDiff(clusters, prob, clus1, clus2, clus3, clus4, xval):
    global INDEX, COUNTER, TRACKER
    try:
        executebase = insertdefault.replace('TABLEPROB', prob)
        executebase = executebase.replace('CATEGORY', clusters[0][0])
        executebase = executebase.replace('CLUSTNUM', str(clusters[0][1]))
        executebase = executebase.replace('NEWID', str(COUNTER))
        COUNTER = COUNTER + 1
        execute1 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute2 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute3 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute4 = executebase.replace('INDEX', str(INDEX))
        INDEX = INDEX + 1
        TRACKER = TRACKER + 1
        execute1 = execute1.replace('PRICE', "'"+str(clusters[clus1][3])+"'")
        execute2 = execute2.replace('PRICE', "'"+str(clusters[clus2][3])+"'")
        execute3 = execute3.replace('PRICE', "'"+str(clusters[clus3][3])+"'")
        execute4 = execute4.replace('PRICE', "'"+str(clusters[clus4][3])+"'")
        sentencebase = sentencediff10.replace('REPLACE', str(clusters[0][1]))
        sentencebase = sentencebase.replace('NUMUNO', str(xval))
        sentence1 = sentencebase.replace('NUMDOS', '1')
        sentence2 = sentencebase.replace('NUMDOS', '2')
        sentence3 = sentencebase.replace('NUMDOS', '3')
        sentence4 = sentencebase.replace('NUMDOS', '4')
        execute1 = execute1.replace('SENTENCE', sentence1)
        execute2 = execute2.replace('SENTENCE', sentence2)
        execute3 = execute3.replace('SENTENCE', sentence3)
        execute4 = execute4.replace('SENTENCE', sentence4)
        doSQL(execute1)
        doSQL(execute2)
        doSQL(execute3)
        doSQL(execute4)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
def equals2Dict(clustnum):
    #When there are two y values each of which is 0.5
    tempbase = sqldictadd6.replace('REPLACE', str(clustnum))
    tempbase = tempbase.replace('PROB', '0.5')
    temp1 = tempbase.replace('NUMB', '1')
    temp2 = tempbase.replace('NUMB', '2')
    return temp1 + temp2

def thirds2Dict(clustnum):
    #When there are 2 z values but one is 0.67 and the other is 0.33
    tempbase = sqldictadd7.replace('REPLACE', str(clustnum))
    temp1 = tempbase.replace('NUMB', '1')
    temp1 = temp1.replace('PROB', '0.67')
    temp2 = tempbase.replace('NUMB', '2')
    temp2 = temp2.replace('PROB', '0.33')
    return temp1 + temp2

def quarts2Dict(clustnum):
    #When there are 2 w values but one is 0.75 and the other is 0.25
    tempbase = sqldictadd8.replace('REPLACE', str(clustnum))
    temp1 = tempbase.replace('NUMB', '1')
    temp1 = temp1.replace('PROB', '0.75')
    temp2 = tempbase.replace('NUMB', '2')
    temp2 = temp2.replace('PROB', '0.25')
    return temp1 + temp2

def thirds3Dict(clustnum):
    #When there are 3 w values all 0.33
    tempbase = sqldictadd8.replace('REPLACE', str(clustnum))
    temp1 = tempbase.replace('NUMB', '1')
    temp1 = temp1.replace('PROB', '0.33')
    temp2 = tempbase.replace('NUMB', '2')
    temp2 = temp2.replace('PROB', '0.33')
    temp3 = tempbase.replace('NUMB', '3')
    temp3 = temp3.replace('PROB', '0.33')
    return temp1 + temp2 + temp3

def halfs3Dict(clustnum):
    #When there are 3 v values one which is 0.5, two 0.25
    tempbase = sqldictadd9.replace('REPLACE', str(clustnum))
    temp1 = tempbase.replace('NUMB', '1')
    temp1 = temp1.replace('PROB', '0.5')
    temp2 = tempbase.replace('NUMB', '2')
    temp2 = temp2.replace('PROB', '0.25')
    temp3 = tempbase.replace('NUMB', '3')
    temp3 = temp3.replace('PROB', '0.25')
    return temp1 + temp2 + temp3

def quarts4Dict(clustnum):
    #When there are 4 z values all 0.25
    tempbase = sqldictadd7.replace('REPLACE', str(clustnum))
    temp1 = tempbase.replace('NUMB', '1')
    temp1 = temp1.replace('PROB', '0.25')
    temp2 = tempbase.replace('NUMB', '2')
    temp2 = temp2.replace('PROB', '0.25')
    temp3 = tempbase.replace('NUMB', '3')
    temp3 = temp3.replace('PROB', '0.25')
    temp4 = tempbase.replace('NUMB', '4')
    temp4 = temp4.replace('PROB', '0.25')
    return temp1 + temp2 + temp3 + temp4
        
def completeTable4(prob, base, dictname):
    global INDEX, COUNTER, TRACKER
    INDEX = 0
    COUNTER = 1
    TRACKER = 0
    try:
        startdict = createdict.replace('DICTNAME', dictname)
        doSQL(startdict)
        finaldict = sqldict + "'"
        #print(CLUSTERS)
        for i in CLUSTERS:
            cluster = getCluster(i, base)
            sentencebasic = sentencediff8.replace('REPLACE', str(cluster[0][1]))
            sentencebasic = sentencebasic.replace('NUMUNO', '1')
            sqlbasic = insertdefault.replace('TABLEPROB', prob)
            sqlbasic = sqlbasic.replace('CATEGORY', cluster[0][0])
            sqlbasic = sqlbasic.replace('CLUSTNUM', str(cluster[0][1]))
            sql1 = sqlbasic.replace('INDEX', str(INDEX))
            id1 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql1 = sql1.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[0][3] == None):
                sql1 = sql1.replace('PRICE', 'NULL')
            else:
                sql1 = sql1.replace('PRICE', "'"+str(cluster[0][3])+"'")
            sentence1 = sentencebasic.replace('NUMDOS', '2')
            sentence1 = sentence1.replace('NUMTRES', '8')
            sentence1 = sentence1.replace('NUMQUAT', '11')
            sentence1 = sentence1.replace('NUMQUIN', '14')
            sql1 = sql1.replace('SENTENCE', sentence1)
            sql2 = sqlbasic.replace('INDEX', str(INDEX))
            id2 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql2 = sql2.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[1][3] == None):
                sql2 = sql2.replace('PRICE', 'NULL')
            else:
                sql2 = sql2.replace('PRICE', "'"+str(cluster[1][3])+"'")
            sentence2 = sentencebasic.replace('NUMDOS', '3')
            sentence2 = sentence2.replace('NUMTRES', '8')
            sentence2 = sentence2.replace('NUMQUAT', '9')
            sentence2 = sentence2.replace('NUMQUIN', '12')
            sql2 = sql2.replace('SENTENCE', sentence2)
            sql3 = sqlbasic.replace('INDEX', str(INDEX))
            id3 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql3 = sql3.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[2][3] == None):
                sql3 = sql3.replace('PRICE', 'NULL')
            else:
                sql3 = sql3.replace('PRICE', "'"+str(cluster[2][3])+"'")
            sentence3 = sentencebasic.replace('NUMDOS', '4')
            sentence3 = sentence3.replace('NUMTRES', '6')
            sentence3 = sentence3.replace('NUMQUAT', '11')
            sentence3 = sentence3.replace('NUMQUIN', '12')
            sql3 = sql3.replace('SENTENCE', sentence3)
            sql4 = sqlbasic.replace('INDEX', str(INDEX))
            id4 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql4 = sql4.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[3][3] == None):
                sql4 = sql4.replace('PRICE', 'NULL')
            else:
                sql4 = sql4.replace('PRICE', "'"+str(cluster[3][3])+"'")
            sentence4 = sentencebasic.replace('NUMDOS', '5')
            sentence4 = sentence4.replace('NUMTRES', '6')
            sentence4 = sentence4.replace('NUMQUAT', '9')
            sentence4 = sentence4.replace('NUMQUIN', '14')
            sql4 = sql4.replace('SENTENCE', sentence1)
            doSQL(sql1)
            doSQL(sql2)
            doSQL(sql3)
            doSQL(sql4)
            temp = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
            temp = temp.replace('PROB', '0.066')
            for j in range(1, 16):
                tempj = temp.replace('NUMB', str(j))
                finaldict = finaldict + tempj
            donthappen1 = cluster[0][0] != cluster[1][0]
            donthappen2 = cluster[0][0] != cluster[2][0]
            donthappen3 = cluster[0][0] != cluster[3][0]
            if (donthappen1 or donthappen2 or donthappen3):
                print("THE THING THAT WASN'T SUPPOSED TO HAPPEN HAPPENED AT:")
                print(cluster[0][1])
            sameAB = cluster[0][3] == cluster[1][3]
            sameAC = cluster[0][3] == cluster[2][3]
            sameAD = cluster[0][3] == cluster[3][3]
            sameBC = cluster[1][3] == cluster[2][3]
            sameBD = cluster[1][3] == cluster[3][3]
            sameCD = cluster[2][3] == cluster[3][3]
            if (sameAB and sameAC and sameAD):
                #OPTION 1
                #All prices are the same, doesn't matter if they're null or not
                sentenceAll(cluster, prob, id1)
                sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
            elif (not sameAB and not sameAC and not sameAD and sameBC and sameBD):
                #OPTION 2
                #A different, BCD the same
                if (cluster[0][3] == None):
                    #Price A is null
                    sentenceAll(cluster, prob, id2)
                    sentenceTen(cluster, prob, id3, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 12, 13, 14)
                elif (cluster[1][3] == None):
                    #Prices B, C, and D are null
                    sentenceAll(cluster, prob, id1)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                else:
                    #The prices are not null
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoThirdDiff(cluster, prob, 2, 0, 3)
                    twoThirdDiff(cluster, prob, 1, 0, 4)
                    twoThirdDiff(cluster, prob, 1, 0, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoQuartDiff(cluster, prob, 1, 0, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + quarts2Dict(cluster[0][1])
            elif (not sameAB and sameAC and sameAD):
                #OPTION 3
                #B different, ACD the same
                if (cluster[0][3] == None):
                    #Prices A, C, and D are null
                    sentenceTen(cluster, prob, id1, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceAll(cluster, prob, id2)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                elif (cluster[1][3] == None):
                    #Price B is null
                    sentenceAll(cluster, prob, id1)
                    sentenceTen(cluster, prob, id3, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 10, 11, 14)
                else:
                    #The prices are not null
                    sentenceTen(cluster, prob, id1, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoThirdDiff(cluster, prob, 2, 1, 2)
                    twoThirdDiff(cluster, prob, 0, 1, 4)
                    twoThirdDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoQuartDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + quarts2Dict(cluster[0][1])
            elif (sameAB and not sameAC and sameAD):
                #OPTION 4
                #C different, ABD the same
                if (cluster[0][3] == None):
                    #Prices A, B, and D are null
                    sentenceTen(cluster, prob, id1, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    sentenceAll(cluster, prob, id3)
                elif (cluster[2][3] == None):
                    #Price C is null
                    sentenceAll(cluster, prob, id1)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 7, 8, 9, 14)
                else:
                    #The prices are not null
                    sentenceTen(cluster, prob, id1, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    twoThirdDiff(cluster, prob, 1, 2, 2)
                    twoThirdDiff(cluster, prob, 0, 2, 3)
                    twoThirdDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoQuartDiff(cluster, prob, 0, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + quarts2Dict(cluster[0][1])
            elif (sameAB and sameAC and not sameAD):
                #OPTION 5
                #D different, ABC the same
                if (cluster[0][3] == None):
                    #Prices A, B and C are null
                    sentenceTen(cluster, prob, id1, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    sentenceAll(cluster, prob, id4)
                elif (cluster[3][3] == None):
                    #Price D is null
                    sentenceAll(cluster, prob, id1)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                else:
                    #The prices are not null
                    sentenceTen(cluster, prob, id1, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id4, 1, 3, 8, 9, 12, 13, 14)
                    twoThirdDiff(cluster, prob, 1, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 3, 3)
                    twoThirdDiff(cluster, prob, 0, 3, 4)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoQuartDiff(cluster, prob, 0, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + quarts2Dict(cluster[0][1])
            elif (sameAB and not sameAC and not sameAD and not sameCD):
                #OPTION 6
                #AB the same, C and D different
                if (cluster[0][3] == None):
                    #Prices A, and B are null
                    sentenceSeven(cluster, prob, id1, 1, 2, 6, 7, 8, 11, 14)
                    sentenceTen(cluster, prob, id3, 1, 4, 5, 6, 9, 10, 11, 12, 13, 14)
                    sentenceTen(cluster, prob, id4, 1, 4, 5, 6, 9, 10, 11, 12, 13, 14)
                    twoDiff(cluster, prob, 2, 3, 2)
                    twoDiff(cluster, prob, 2, 3, 3)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 2, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1])
                elif (cluster[2][3] == None):
                    #Price C is null
                    sentenceTen(cluster, prob, id1, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 7, 8, 9, 14)
                    twoDiff(cluster, prob, 1, 3, 2)
                    twoDiff(cluster, prob, 0, 3, 3)
                    twoThirdDiff(cluster, prob, 0, 3, 4)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoThirdDiff(cluster, prob, 0, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[3][3] == None):
                    #Price D is null
                    sentenceTen(cluster, prob, id1, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoDiff(cluster, prob, 1, 2, 2)
                    twoDiff(cluster, prob, 0, 2, 3)
                    twoThirdDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoThirdDiff(cluster, prob, 0, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id1, 1, 2, 6, 7, 8, 11, 14)
                    threeThirdDiff(cluster, prob, 1, 2, 3, 2)
                    threeThirdDiff(cluster, prob, 1, 2, 3, 3)
                    twoThirdDiff(cluster, prob, 0, 3, 4)
                    twoThirdDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeHalfDiff(cluster, prob, 0, 2, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + halfs3Dict(cluster[0][1])
            elif (sameAB and not sameAC and sameCD):
                #OPTION 7
                #AB the same, different from CD the same
                if (cluster[0][3] == None):
                    #Prices A, and B are null
                    sentenceSeven(cluster, prob, id1, 1, 2, 6, 7, 8, 11, 14)
                    sentenceAll(cluster, prob, id3)
                    sentenceTen(cluster, prob, id4, 1, 4, 5, 6, 9, 10, 11, 12, 13, 14)
                elif (cluster[2][3] == None):
                    #Prices C, and D are null
                    sentenceAll(cluster, prob, id1)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id1, 1, 2, 6, 7, 8, 11, 12)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoThirdDiff(cluster, prob, 2, 1, 2)
                    twoThirdDiff(cluster, prob, 2, 0, 3)
                    twoThirdDiff(cluster, prob, 0, 3, 4)
                    twoThirdDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
            elif (not sameAB and not sameAC and not sameAD and not sameBC and sameCD):
                #OPTION 8
                #CD the same, A and B different
                if (cluster[0][3] == None):
                    #Price A is null
                    sentenceSeven(cluster, prob, id2, 1, 3, 6, 7, 8, 9, 12)
                    sentenceTen(cluster, prob, id3, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 12, 13, 14)
                    twoThirdDiff(cluster, prob, 2, 1, 2)
                    twoDiff(cluster, prob, 1, 3, 4)
                    twoDiff(cluster, prob, 1, 2, 5)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoThirdDiff(cluster, prob, 2, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[1][3] == None):
                    #Price B is null
                    sentenceSeven(cluster, prob, id1, 1, 2, 6, 7, 8, 11, 14)
                    sentenceTen(cluster, prob, id3, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 10, 11, 14)
                    twoThirdDiff(cluster, prob, 2, 0, 3)
                    twoDiff(cluster, prob, 0, 3, 4)
                    twoDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoThirdDiff(cluster, prob, 2, 0, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[2][3] == None):
                    #Prices C, and D are null
                    sentenceTen(cluster, prob, id1, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoDiff(cluster, prob, 0, 1, 4)
                    twoDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1])
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoThirdDiff(cluster, prob, 2, 1, 2)
                    twoThirdDiff(cluster, prob, 2, 0, 3)
                    threeThirdDiff(cluster, prob, 0, 1, 3, 4)
                    threeThirdDiff(cluster, prob, 0, 1, 2, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeHalfDiff(cluster, prob, 2, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + halfs3Dict(cluster[0][1])
            elif (not sameAB and sameAC and not sameAD and not sameBD):
                #OPTION 9
                #AC the same, B and D different
                if (cluster[0][3] == None):
                    #Prices A, and C are null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 9, 10, 11, 14)
                    sentenceTen(cluster, prob, id2, 1, 3, 5, 6, 7, 8, 9, 12, 13, 14)
                    sentenceTen(cluster, prob, id4, 1, 3, 5, 6, 7, 8, 9, 12, 13, 14)
                    twoDiff(cluster, prob, 1, 3, 2)
                    twoDiff(cluster, prob, 1, 3, 4)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 1, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1])
                elif (cluster[1][3] == None):
                    #Price B is null
                    sentenceTen(cluster, prob, id1, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 10, 11, 14)
                    twoDiff(cluster, prob, 2, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 3, 3)
                    twoDiff(cluster, prob, 0, 3, 4)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoThirdDiff(cluster, prob, 0, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[3][3] == None):
                    #Price D is null
                    sentenceTen(cluster, prob, id1, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoDiff(cluster, prob, 1, 2, 2)
                    twoDiff(cluster, prob, 0, 1, 4)
                    twoThirdDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoThirdDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 9, 10, 11, 14)
                    threeThirdDiff(cluster, prob, 1, 2, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 3, 3)
                    threeThirdDiff(cluster, prob, 0, 1, 3, 4)
                    twoThirdDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeHalfDiff(cluster, prob, 0, 1, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + halfs3Dict(cluster[0][1])
            elif (not sameAB and sameAC and sameBD):
                #OPTION 10
                #AC the same, different from BD the same
                if (cluster[0][3] == None):
                    #Prices A, and C are null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 9, 10, 11, 14)
                    sentenceAll(cluster, prob, id2)
                    sentenceTen(cluster, prob, id4, 1, 3, 5, 6, 7, 8, 9, 12, 13, 14)
                elif (cluster[1][3] == None):
                    #Prices B, and D are null
                    sentenceAll(cluster, prob, id1)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    sentenceTen(cluster, prob, id3, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    twoThirdDiff(cluster, prob, 1, 2, 2)
                    twoThirdDiff(cluster, prob, 0, 3, 3)
                    twoThirdDiff(cluster, prob, 1, 0, 4)
                    twoThirdDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
            elif (not sameAB and not sameAC and not sameAD and not sameBC and sameBD):
                #OPTION 11
                #BD the same, A and C different
                if (cluster[0][3] == None):
                    #Price A is null
                    sentenceTen(cluster, prob, id2, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 12, 13, 14)
                    twoThirdDiff(cluster, prob, 1, 2, 2)
                    twoDiff(cluster, prob, 2, 3, 3)
                    twoDiff(cluster, prob, 1, 2, 5)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoThirdDiff(cluster, prob, 1, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[1][3] == None):
                    #Prices B, and D are null
                    sentenceTen(cluster, prob, id1, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    sentenceTen(cluster, prob, id3, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    twoDiff(cluster, prob, 0, 2, 3)
                    twoDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 0, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1])
                elif (cluster[2][3] == None):
                    #Price C is null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 9, 10, 11, 14)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 7, 8, 9, 14)
                    twoDiff(cluster, prob, 0, 3, 3)
                    twoThirdDiff(cluster, prob, 1, 0, 4)
                    twoDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoThirdDiff(cluster, prob, 1, 0, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    twoThirdDiff(cluster, prob, 1, 2, 2)
                    threeThirdDiff(cluster, prob, 0, 2, 3, 3)
                    twoThirdDiff(cluster, prob, 1, 0, 4)
                    threeThirdDiff(cluster, prob, 0, 1, 2, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeHalfDiff(cluster, prob, 1, 0, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + halfs3Dict(cluster[0][1])
            elif (not sameAB and not sameAC and sameAD and not sameBC):
                #OPTION 12
                #AD the same, B and C different
                if (cluster[0][3] == None):
                    #Prices A, and D are null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 11, 12, 13, 14)
                    sentenceTen(cluster, prob, id2, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12)
                    sentenceTen(cluster, prob, id3, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12)
                    twoDiff(cluster, prob, 1, 2, 2)
                    twoDiff(cluster, prob, 1, 2, 5)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    twoDiff(cluster, prob, 1, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1])
                elif (cluster[1][3] == None):
                    #Price B is null
                    sentenceTen(cluster, prob, id1, 1, 2, 4, 6, 7, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 10, 11, 14)
                    twoDiff(cluster, prob, 2, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 2, 3)
                    twoDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoThirdDiff(cluster, prob, 0, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[2][3] == None):
                    #Price C is null
                    sentenceTen(cluster, prob, id1, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 7, 8, 9, 14)
                    twoDiff(cluster, prob, 1, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 1, 4)
                    twoDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoThirdDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 11, 12, 13, 14)
                    threeThirdDiff(cluster, prob, 1, 2, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 2, 3)
                    twoThirdDiff(cluster, prob, 0, 1, 4)
                    threeThirdDiff(cluster, prob, 0, 1, 2, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeHalfDiff(cluster, prob, 0, 1, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + halfs3Dict(cluster[0][1])
            elif (not sameAB and not sameAC and sameAD and sameBC):
                #OPTION 13
                #AD the same, different from BC the same
                if (cluster[0][3] == None):
                    #Prices A, and D are null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 11, 12, 13, 14)
                    sentenceAll(cluster, prob, id2)
                    sentenceTen(cluster, prob, id3, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12)
                elif (cluster[1][3] == None):
                    #Prices B, and C are null
                    sentenceAll(cluster, prob, id1)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    sentenceTen(cluster, prob, id4, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    twoThirdDiff(cluster, prob, 1, 3, 2)
                    twoThirdDiff(cluster, prob, 0, 2, 3)
                    twoThirdDiff(cluster, prob, 0, 1, 4)
                    twoThirdDiff(cluster, prob, 1, 0, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 1, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
            elif (not sameAB and not sameAC and not sameAD and sameBC and not sameBD):
                #OPTION 14
                #BC the same, A and D different
                if (cluster[0][3] == None):
                    #Price A is null
                    sentenceTen(cluster, prob, id2, 1, 3, 5, 6, 7, 8, 9, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 12, 13, 14)
                    twoThirdDiff(cluster, prob, 1, 3, 2)
                    twoDiff(cluster, prob, 2, 3, 3)
                    twoDiff(cluster, prob, 1, 3, 4)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoThirdDiff(cluster, prob, 1, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                elif (cluster[1][3] == None):
                    #Prices B, and C are null
                    sentenceTen(cluster, prob, id1, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    sentenceTen(cluster, prob, id4, 1, 2, 5, 6, 7, 8, 9, 10, 11, 14)
                    twoDiff(cluster, prob, 0, 3, 3)
                    twoDiff(cluster, prob, 0, 3, 4)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 0, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1])
                elif (cluster[3][3] == None):
                    #Price D is null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 11, 12, 13, 14)
                    sentenceTen(cluster, prob, id2, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoDiff(cluster, prob, 0, 2, 3)
                    twoDiff(cluster, prob, 0, 1, 4)
                    twoThirdDiff(cluster, prob, 1, 0, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoThirdDiff(cluster, prob, 1, 0, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1])
                else:
                    #The prices are not null
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    twoThirdDiff(cluster, prob, 1, 3, 2)
                    threeThirdDiff(cluster, prob, 0, 2, 3, 3)
                    threeThirdDiff(cluster, prob, 0, 1, 3, 4)
                    twoThirdDiff(cluster, prob, 1, 0, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    threeHalfDiff(cluster, prob, 1, 0, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + halfs3Dict(cluster[0][1])
            elif (not sameAB and not sameAC and not sameAD and not sameBC and not sameBD and not sameCD):
                #OPTION 15
                #All different
                if (cluster[0][3] == None):
                    #Price A is null
                    sentenceSeven(cluster, prob, id2, 1, 3, 6, 7, 8, 9, 12)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 12, 13, 14)
                    threeThirdDiff(cluster, prob, 1, 2, 3, 2)
                    twoDiff(cluster, prob, 2, 3, 3)
                    twoDiff(cluster, prob, 1, 3, 4)
                    twoDiff(cluster, prob, 1, 2, 5)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeThirdDiff(cluster, prob, 1, 2, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1])
                elif (cluster[1][3] == None):
                    #Price B is null
                    sentenceSeven(cluster, prob, id1, 1, 2, 3, 6, 7, 8, 14)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 9, 10, 11, 14)
                    twoDiff(cluster, prob, 2, 3, 2)
                    threeThirdDiff(cluster, prob, 0, 2, 3, 3)
                    twoDiff(cluster, prob, 0, 3, 4)
                    twoDiff(cluster, prob, 0, 2, 5)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    threeThirdDiff(cluster, prob, 0, 2, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1])
                elif (cluster[2][3] == None):
                    #Price C is null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 9, 10, 11, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 12, 13, 14)
                    sentenceSeven(cluster, prob, id4, 1, 5, 6, 7, 8, 9, 14)
                    twoDiff(cluster, prob, 1, 3, 2)
                    twoDiff(cluster, prob, 0, 3, 3)
                    threeThirdDiff(cluster, prob, 0, 1, 3, 4)
                    twoDiff(cluster, prob, 0, 1, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    threeThirdDiff(cluster, prob, 0, 1, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1])
                elif (cluster[3][3] == None):
                    #Price D is null
                    sentenceSeven(cluster, prob, id1, 1, 2, 8, 11, 12, 13, 14)
                    sentenceSeven(cluster, prob, id2, 1, 3, 8, 9, 10, 11, 12)
                    sentenceSeven(cluster, prob, id3, 1, 4, 6, 7, 8, 11, 12)
                    twoDiff(cluster, prob, 1, 2, 2)
                    twoDiff(cluster, prob, 0, 2, 3)
                    twoDiff(cluster, prob, 0, 1, 4)
                    threeThirdDiff(cluster, prob, 0, 1, 2, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    threeThirdDiff(cluster, prob, 0, 1, 2, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1])
                else:
                    #The prices are not null
                    threeThirdDiff(cluster, prob, 1, 2, 3, 2)
                    threeThirdDiff(cluster, prob, 0, 2, 3, 3)
                    threeThirdDiff(cluster, prob, 0, 1, 3, 4)
                    threeThirdDiff(cluster, prob, 0, 1, 2, 5)
                    twoDiff(cluster, prob, 0, 1, 6)
                    twoDiff(cluster, prob, 0, 1, 7)
                    twoDiff(cluster, prob, 2, 3, 7)
                    twoDiff(cluster, prob, 2, 3, 8)
                    twoDiff(cluster, prob, 0, 2, 9)
                    twoDiff(cluster, prob, 0, 2, 10)
                    twoDiff(cluster, prob, 1, 3, 10)
                    twoDiff(cluster, prob, 1, 3, 11)
                    twoDiff(cluster, prob, 0, 3, 12)
                    twoDiff(cluster, prob, 0, 3, 13)
                    twoDiff(cluster, prob, 1, 2, 13)
                    twoDiff(cluster, prob, 1, 2, 14)
                    fourQaurDiff(cluster, prob, 0, 1, 2, 3, 15)
                    finaldict = finaldict + equals2Dict(cluster[0][1]) + thirds3Dict(cluster[0][1]) + quarts4Dict(cluster[0][1])
            else:
                #SHOULD NOT HAPPEN, but yknow how coding is
                print('OH GOD NO. LOOK AT CLUSTER:')
                print(cluster[0][1])
            if (TRACKER > 100):
                TRACKER = 0
                sys.stdout.write('.')
                sys.stdout.flush()
        finaldict = finaldict + "') WHERE name='DICTNAME';"
        finaldict = finaldict.replace('DICTNAME', dictname)
        doSQL(finaldict)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [4]:
def completeTable3(prob, base, dictname):
    global INDEX, COUNTER, TRACKER
    INDEX = 0
    COUNTER = 1
    TRACKER = 0
    try:
        startdict = createdict.replace('DICTNAME', dictname)
        doSQL(startdict)
        finaldict = sqldict + "'"
        for i in CLUSTERS:
            cluster = getCluster(i, base)
            sentencebasic = sentencediff5.replace('REPLACE', str(cluster[0][1]))
            sentencebasic = sentencebasic.replace('NUMUNO', '1')
            sqlbasic = insertdefault.replace('TABLEPROB', prob)
            sqlbasic = sqlbasic.replace('CATEGORY', cluster[0][0])
            sqlbasic = sqlbasic.replace('CLUSTNUM', str(cluster[0][1]))
            sql1 = sqlbasic.replace('INDEX', str(INDEX))
            id1 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql1 = sql1.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[0][3] == None):
                sql1 = sql1.replace('PRICE', 'NULL')
            else:
                sql1 = sql1.replace('PRICE', "'"+str(cluster[0][3])+"'")
            sentence1 = sentencebasic.replace('NUMDOS', '2')
            sql1 = sql1.replace('SENTENCE', sentence1)
            sql2 = sqlbasic.replace('INDEX', str(INDEX))
            id2 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql2 = sql2.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[1][3] == None):
                sql2 = sql2.replace('PRICE', 'NULL')
            else:
                sql2 = sql2.replace('PRICE', "'"+str(cluster[1][3])+"'")
            sentence2 = sentencebasic.replace('NUMDOS', '3')
            sql2 = sql2.replace('SENTENCE', sentence2)
            sql3 = sqlbasic.replace('INDEX', str(INDEX))
            id3 = INDEX
            INDEX = INDEX + 1
            TRACKER = TRACKER + 1
            sql3 = sql3.replace('NEWID', str(COUNTER))
            COUNTER = COUNTER + 1
            if (cluster[2][3] == None):
                sql3 = sql3.replace('PRICE', 'NULL')
            else:
                sql3 = sql3.replace('PRICE', "'"+str(cluster[2][3])+"'")
            sentence1 = sentencebasic.replace('NUMDOS', '4')
            sql3 = sql3.replace('SENTENCE', sentence1)
            doSQL(sql1)
            doSQL(sql2)
            doSQL(sql3)
            temp = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
            temp = temp.replace('NUMB', '1')
            temp = temp.replace('PROB', '0.2')
            finaldict = finaldict + temp
            donthappen1 = cluster[0][0] != cluster[1][0]
            donthappen2 = cluster[0][0] != cluster[2][0]
            if (donthappen1 or donthappen2):
                print("THE THING THAT WASN'T SUPPOSED TO HAPPEN HAPPENED AT:")
                print(cluster[0][1])
            same1 = cluster[0][3] == cluster[1][3]
            same2 = cluster[0][3] == cluster[2][3]
            same3 = cluster[1][3] == cluster[2][3]
            if (same1 and same2 and same3):
                #all prices are the same
                sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                sentence = sentence.replace('NUMUNO', '1')
                sentence = sentence.replace('NUMDOS', '2')
                sentence = sentence.replace('NUMTRES', '3')
                sentence = sentence.replace('NUMQUAT', '4')
                sentence = sentence.replace('NUMQUIN', '5')
                execute = sqlsentence.replace('TABLEPROB', prob)
                execute = execute.replace('INSERTHERE', sentence)
                execute = execute.replace('INDEX', str(id1))
                #print(execute)
                #print('\n')
                doSQL(execute)
                #DICT
                tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                tempgeneral = tempgeneral.replace('PROB', '0.2')
                temp1 = tempgeneral.replace('NUMB', '2')
                temp2 = tempgeneral.replace('NUMB', '3')
                temp3 = tempgeneral.replace('NUMB', '4')
                temp4 = tempgeneral.replace('NUMB', '5')
                finaldict = finaldict + temp1 + temp2 + temp3 + temp4
            elif (not same1 and not same2 and same3):
                #The first price is different, the last two are the same
                if (cluster[0][3] == None):
                    #A BC, AB C, ABC
                    sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    sentence = sentence.replace('NUMQUAT', '4')
                    sentence = sentence.replace('NUMQUIN', '5')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    doSQL(execute)
                    #AC B
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '3')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id3))
                    doSQL(execute)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4
                elif (cluster[1][3] == None):
                    #A BC
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    doSQL(execute)
                    #AC B, AB C, ABC
                    sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    sentence = sentence.replace('NUMQUAT', '4')
                    sentence = sentence.replace('NUMQUIN', '5')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4
                else:
                    # A BC
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    doSQL(execute)
                    #AC B
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '3')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AB C
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '4')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #ABC
                    sentencedefault = sentencediff10.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #DICT STUFF
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    tempgen3 = sqldictadd7.replace('REPLACE', str(cluster[0][1]))
                    temp7 = tempgen3.replace('NUMB', '1')
                    temp7 = temp7.replace('PROB', '0.33')
                    temp8 = tempgen3.replace('NUMB', '2')
                    temp8 = temp8.replace('PROB', '0.67')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6 + temp7 + temp8
            elif (same1 and not same2 and not same3):
                #The first two prices are the same, the last one is different
                if (cluster[0][3] == None):
                    #A BC, AC B, ABC
                    sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    sentence = sentence.replace('NUMQUAT', '4')
                    sentence = sentence.replace('NUMQUIN', '5')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id3))
                    doSQL(execute)
                    #AB C
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4
                elif (cluster[2][3] == None):
                    #A BC
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    #AC B, AB C, ABC
                    sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    sentence = sentence.replace('NUMQUAT', '4')
                    sentence = sentence.replace('NUMQUIN', '5')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4
                else:
                    #A BC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '2')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AC B
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '3')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AB C
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #ABC
                    sentencedefault = sentencediff10.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    tempgen3 = sqldictadd7.replace('REPLACE', str(cluster[0][1]))
                    temp7 = tempgen3.replace('NUMB', '1')
                    temp7 = temp7.replace('PROB', '0.67')
                    temp8 = tempgen3.replace('NUMB', '2')
                    temp8 = temp8.replace('PROB', '0.33')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6 + temp7 + temp8
            elif (not same1 and same2 and not same3):
                #The first and last prices are the same, the middle is different
                if (cluster[0][3] == None):
                    #A BC, AB C, ABC
                    sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    sentence = sentence.replace('NUMQUAT', '4')
                    sentence = sentence.replace('NUMQUIN', '5')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    doSQL(execute)
                    #AC B
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4
                elif (cluster[1][3] == None):
                    #A BC
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id3))
                    #AC B, AB C, ABC
                    sentence = sentencediff8.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    sentence = sentence.replace('NUMQUAT', '4')
                    sentence = sentence.replace('NUMQUIN', '5')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4
                else:
                    #A BC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '2')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AC B
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #AB C
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '4')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #ABC
                    sentencedefault = sentencediff10.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    tempgen3 = sqldictadd7.replace('REPLACE', str(cluster[0][1]))
                    temp7 = tempgen3.replace('NUMB', '1')
                    temp7 = temp7.replace('PROB', '0.67')
                    temp8 = tempgen3.replace('NUMB', '2')
                    temp8 = temp8.replace('PROB', '0.33')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6 + temp7 + temp8
            else:
                #The three prices are all different
                if (cluster[0][3] == None):
                    #A BC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '2')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AC B
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '3')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id3))
                    doSQL(execute)
                    #AB C
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '3')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    doSQL(execute)
                    #ABC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6
                elif (cluster[1][3] == None):
                    #A BC
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id3))
                    doSQL(execute)
                    #AC B
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '3')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AB C
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '4')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #ABC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6
                elif (cluster[2][3] == None):
                    #A BC
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id2))
                    doSQL(execute)
                    #AC B
                    sentence = sentencediff6.replace('REPLACE', str(cluster[0][1]))
                    sentence = sentence.replace('NUMUNO', '1')
                    sentence = sentence.replace('NUMDOS', '2')
                    sentence = sentence.replace('NUMTRES', '3')
                    execute = sqlsentence.replace('TABLEPROB', prob)
                    execute = execute.replace('INSERTHERE', sentence)
                    execute = execute.replace('INDEX', str(id1))
                    doSQL(execute)
                    #AB C
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '4')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #ABC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6
                else:
                    #A BC
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '2')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AC B
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '3')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #AB C
                    sentencedefault = sentencediff9.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '4')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    doSQL(execute1)
                    doSQL(execute2)
                    #ABC
                    sentencedefault = sentencediff10.replace('REPLACE', str(cluster[0][1]))
                    sentencedefault = sentencedefault.replace('NUMUNO', '5')
                    sentence1 = sentencedefault.replace('NUMDOS', '1')
                    sentence2 = sentencedefault.replace('NUMDOS', '2')
                    sentence3 = sentencedefault.replace('NUMDOS', '3')
                    executebase = insertdefault.replace('TABLEPROB', prob)
                    executebase = executebase.replace('CATEGORY', cluster[0][0])
                    executebase = executebase.replace('CLUSTNUM', str(cluster[0][1]))
                    executebase = executebase.replace('NEWID', str(COUNTER))
                    COUNTER = COUNTER + 1
                    execute1 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute2 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute3 = executebase.replace('INDEX', str(INDEX))
                    INDEX = INDEX + 1
                    TRACKER = TRACKER + 1
                    execute1 = execute1.replace('PRICE', "'"+str(cluster[0][3])+"'")
                    execute2 = execute2.replace('PRICE', "'"+str(cluster[1][3])+"'")
                    execute3 = execute3.replace('PRICE', "'"+str(cluster[2][3])+"'")
                    execute1 = execute1.replace('SENTENCE', sentence1)
                    execute2 = execute2.replace('SENTENCE', sentence2)
                    execute3 = execute3.replace('SENTENCE', sentence3)
                    doSQL(execute1)
                    doSQL(execute2)
                    doSQL(execute3)
                    #DICT
                    tempgeneral = sqldictadd5.replace('REPLACE', str(cluster[0][1]))
                    tempgeneral = tempgeneral.replace('PROB', '0.2')
                    temp1 = tempgeneral.replace('NUMB', '2')
                    temp2 = tempgeneral.replace('NUMB', '3')
                    temp3 = tempgeneral.replace('NUMB', '4')
                    temp4 = tempgeneral.replace('NUMB', '5')
                    tempgen2 = sqldictadd6.replace('REPLACE', str(cluster[0][1]))
                    temp5 = tempgen2.replace('NUMB', '1')
                    temp5 = temp5.replace('PROB', '0.5')
                    temp6 = tempgen2.replace('NUMB', '2')
                    temp6 = temp6.replace('PROB', '0.5')
                    tempgen3 = sqldictadd7.replace('REPLACE', str(cluster[0][1]))
                    temp7 = tempgen3.replace('NUMB', '1')
                    temp7 = temp7.replace('PROB', '0.33')
                    temp8 = tempgen3.replace('NUMB', '2')
                    temp8 = temp8.replace('PROB', '0.33')
                    temp9 = tempgen3.replace('NUMB', '3')
                    temp9 = temp9.replace('PROB', '0.33')
                    finaldict = finaldict + temp1 + temp2 + temp3 + temp4 + temp5 + temp6 + temp7 + temp8 + temp9
            if (TRACKER > 100):
                TRACKER = 0
                sys.stdout.write('.')
                sys.stdout.flush()
        finaldict = finaldict + "') WHERE name='DICTNAME';"
        finaldict = finaldict.replace('DICTNAME', dictname)
        doSQL(finaldict)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [33]:
if __name__ == '__main__':
    connect_pg(configname='database.ini')
    getIDs(sqlbase, 'wdc_eng_ex_s2n4000_base')
    completeTable2('wdc_eng_ex_s2n4000_prob', 'wdc_eng_ex_s2n4000_base', 's2n4000')
    #completeTable3('wdc_eng_ex_s3n4000_prob', 'wdc_eng_ex_s3n4000_base', 's3n4000')
    #completeTable4('wdc_eng_ex_s4n4000_prob', 'wdc_eng_ex_s4n4000_base', 's4n4000')
    close_pg()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.18 (Ubuntu 12.18-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0, 64-bit',)
................................................................................Database connection closed.
